<a href="https://colab.research.google.com/github/Satwikram/Machine-Learning-Notebooks/blob/master/SGDClassifier/Heart_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"satwikram","key":"06c98063bdd0e68efaf2312bfe2c72df"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d danimal/heartdiseaseensembleclassifier

  0% 0.00/8.80k [00:00<?, ?B/s]
100% 8.80k/8.80k [00:00<00:00, 7.80MB/s]


In [ ]:
!unzip /content/heartdiseaseensembleclassifier.zip

Archive:  /content/heartdiseaseensembleclassifier.zip
  inflating: Heart_Disease_Data.csv  
  inflating: case1.R                 
  inflating: hd.txt                  


Importing Libraries

In [ ]:
"""
@author : Satwik Ram K

"""
import numpy as np
import pandas as pd

Relevant Information

This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them.

In particular, the Cleveland Clinic Foundation (cleveland.data) database is the only one that has been used by ML researchers to this date.

The "pred_attribute" field refers to the presence of heart disease in the patient.

Experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0).

Complete attribute documentation

age: age in years

sex: (1 = male; 0 = female)

cp: chest pain type

Value 1: typical angina

Value 2: atypical angina

Value 3: non-anginal pain

Value 4: asymptomatic

trestbps: resting blood pressure (in mm Hg on admission to the hospital)

chol: serum cholestoral in mg/dl

fbs: (fasting blood sugar > 120 mg/dl)

1 = true

0 = false

restecg: (resting electrocardiographic results)

Value 0: normal

Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)

Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

thalach: maximum heart rate achieved

exang: exercise induced angina

1 = yes

0 = no

oldpeak: ST depression induced by exercise relative to rest

slope: the slope of the peak exercise ST segment

Value 1: upsloping

Value 2: flat

Value 3: downsloping

ca: number of major vessels (0-3) colored by flourosopy

thal: thalium heart scan

3 = normal (no cold spots)

6 = fixed defect (cold spots during rest and exercise)

7 = reversible defect (when cold spots only appear during exercise)

pred_attribute: (the predicted attribute) diagnosis of heart disease 
(angiographic disease status)

Value 0: < 50% diameter narrowing

Value 1: > 50% diameter narrowing (in any major vessel: attributes 59 through 68 are vessels)

In [ ]:
dataset = pd.read_csv('/content/Heart_Disease_Data.csv')

In [ ]:
dataset.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slop,ca,thal,pred_attribute
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
5,56,1,2,120,236,0,0,178,0,0.8,1,0,3,0
6,62,0,4,140,268,0,2,160,0,3.6,3,2,3,3
7,57,0,4,120,354,0,0,163,1,0.6,1,0,3,0
8,63,1,4,130,254,0,2,147,0,1.4,2,1,7,2
9,53,1,4,140,203,1,2,155,1,3.1,3,0,7,1


In [ ]:
dataset.count()

age               303
sex               303
cp                303
trestbps          303
chol              303
fbs               303
restecg           303
thalach           303
exang             303
oldpeak           303
slop              303
ca                303
thal              303
pred_attribute    303
dtype: int64

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             303 non-null    int64  
 1   sex             303 non-null    int64  
 2   cp              303 non-null    int64  
 3   trestbps        303 non-null    int64  
 4   chol            303 non-null    int64  
 5   fbs             303 non-null    int64  
 6   restecg         303 non-null    int64  
 7   thalach         303 non-null    int64  
 8   exang           303 non-null    int64  
 9   oldpeak         303 non-null    float64
 10  slop            303 non-null    int64  
 11  ca              303 non-null    object 
 12  thal            303 non-null    object 
 13  pred_attribute  303 non-null    int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 33.3+ KB


In [ ]:
for column in dataset.columns:
  print(column,"Unique Values are:",dataset[column].unique())
  print("---"*30)

age Unique Values are: [63 67 37 41 56 62 57 53 44 52 48 54 49 64 58 60 50 66 43 40 69 59 42 55
 61 65 71 51 46 45 39 68 47 34 35 29 70 77 38 74 76]
------------------------------------------------------------------------------------------
sex Unique Values are: [1 0]
------------------------------------------------------------------------------------------
cp Unique Values are: [1 4 3 2]
------------------------------------------------------------------------------------------
trestbps Unique Values are: [145 160 120 130 140 172 150 110 132 117 135 112 105 124 125 142 128 170
 155 104 180 138 108 134 122 115 118 100 200  94 165 102 152 101 126 174
 148 178 158 192 129 144 123 136 146 106 156 154 114 164]
------------------------------------------------------------------------------------------
chol Unique Values are: [233 286 229 250 204 236 268 354 254 203 192 294 256 263 199 168 239 275
 266 211 283 284 224 206 219 340 226 247 167 230 335 234 177 276 353 243
 225 302 212 330 175 417

In [ ]:
dataset.isna().sum()

age               0
sex               0
cp                0
trestbps          0
chol              0
fbs               0
restecg           0
thalach           0
exang             0
oldpeak           0
slop              0
ca                0
thal              0
pred_attribute    0
dtype: int64

In [ ]:
dataset['thal'].replace('?', np.nan, inplace = True)

In [ ]:
dataset['ca'].replace('?', np.nan, inplace = True)

In [ ]:
dataset.isna().sum()

age               0
sex               0
cp                0
trestbps          0
chol              0
fbs               0
restecg           0
thalach           0
exang             0
oldpeak           0
slop              0
ca                4
thal              2
pred_attribute    0
dtype: int64

In [ ]:
dataset.dropna(inplace = True)

In [ ]:
dataset.isna().sum()

age               0
sex               0
cp                0
trestbps          0
chol              0
fbs               0
restecg           0
thalach           0
exang             0
oldpeak           0
slop              0
ca                0
thal              0
pred_attribute    0
dtype: int64

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 301
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             297 non-null    int64  
 1   sex             297 non-null    int64  
 2   cp              297 non-null    int64  
 3   trestbps        297 non-null    int64  
 4   chol            297 non-null    int64  
 5   fbs             297 non-null    int64  
 6   restecg         297 non-null    int64  
 7   thalach         297 non-null    int64  
 8   exang           297 non-null    int64  
 9   oldpeak         297 non-null    float64
 10  slop            297 non-null    int64  
 11  ca              297 non-null    object 
 12  thal            297 non-null    object 
 13  pred_attribute  297 non-null    int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 34.8+ KB


In [ ]:
for column in dataset.columns:
  if dataset[column].dtype == 'object':
    print(column, "Object exist")
    print(column,"Unique Values are:",dataset[column].unique())


ca Object exist
ca Unique Values are: ['0' '3' '2' '1']
thal Object exist
thal Unique Values are: ['6' '3' '7']


Values are in single quotes, we need to fix it!!

In [ ]:
replace_num = {'ca': {'0': 0, '1': 1, '2': 2, '3': 3},
               'thal': {'6': 6, '3': 3, '7': 7}}

In [ ]:
dataset.replace(replace_num, inplace = True)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 301
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             297 non-null    int64  
 1   sex             297 non-null    int64  
 2   cp              297 non-null    int64  
 3   trestbps        297 non-null    int64  
 4   chol            297 non-null    int64  
 5   fbs             297 non-null    int64  
 6   restecg         297 non-null    int64  
 7   thalach         297 non-null    int64  
 8   exang           297 non-null    int64  
 9   oldpeak         297 non-null    float64
 10  slop            297 non-null    int64  
 11  ca              297 non-null    int64  
 12  thal            297 non-null    int64  
 13  pred_attribute  297 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 34.8 KB


In [ ]:
dataset["pred_attribute"].replace(inplace=True, value=[1, 1, 1, 1], to_replace=[1, 2, 3, 4])

In [ ]:
dataset.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slop,ca,thal,pred_attribute
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,54.542088,0.676768,3.158249,131.693603,247.350168,0.144781,0.996633,149.599327,0.326599,1.055556,1.602694,0.676768,4.730640,0.461279
std,9.049736,0.468500,0.964859,17.762806,51.997583,0.352474,0.994914,22.941562,0.469761,1.166123,0.618187,0.938965,1.938629,0.499340
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,243.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,276.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,1.000000


Taking X and Y

In [ ]:
X = dataset.iloc[:, :13].values

In [ ]:
y = dataset.iloc[:, 13:].values

Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

Normalizing the Data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Fitting Classifer and finding the best!

SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(random_state = 42)

In [ ]:
sgd.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
y_pred = sgd.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

array([[38, 11],
       [11, 30]])

In [ ]:
score = accuracy_score(y_test, y_pred)

In [ ]:
score*100

75.55555555555556

Tuning SGD

In [ ]:
from sklearn.model_selection import GridSearchCV
 

In [ ]:
param_grid = {
    'learning_rate' : ['optimal', 'constant', 'invscaling', 'adaptive'],
    'early_stopping' : [True, False],
    'warm_start' : [True, False],
    'shuffle' : [True, False],
    'loss' : ['hinge', 'log', 'squared_hinge', 'modified_huber'],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "none"],
    'max_iter' : [10, 100, 500, 1000]
    }

In [ ]:
clf = GridSearchCV(sgd, param_grid = param_grid)

In [ ]:
try:
  clf.fit(X_train, y_train)
except:
  pass

In [ ]:
print(clf.best_score_)

0.8212543554006968


In [ ]:
print(clf.best_estimator_)

SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=10,
              n_iter_no_change=5, n_jobs=None, penalty='none', power_t=0.5,
              random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=True)


In [ ]:
sgd = SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=10,
              n_iter_no_change=5, n_jobs=None, penalty='none', power_t=0.5,
              random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=True)

In [ ]:
sgd.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=10,
              n_iter_no_change=5, n_jobs=None, penalty='none', power_t=0.5,
              random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=True)

In [ ]:
y_pred = sgd.predict(X_test)

In [ ]:
score = accuracy_score(y_test, y_pred)

In [ ]:
score * 100

86.66666666666667